In [ ]:
cd "./dataset/pre_process_data"

In [ ]:
pip install reverse_geocoder pytz timezonefinder --user

In [ ]:
import os

files_list = []

for root, dirs, files in os.walk("./01"):
    for file in files:
        if file.endswith(".json"):
             files_list.append(str(os.path.join(root, file)))

In [ ]:
import json
import pandas as pd

geos = pd.DataFrame([])

geo_cnt = 0
place_cnt = 0
user_loc_cnt = 0

for file_name in files_list:
    temp = pd.read_json(file_name, orient='records',convert_dates=False)
    geos = pd.concat([geos,temp], ignore_index=True)

In [ ]:
def fill_coordinates(row):
    try:
        coordinates = row['place']['bounding_box']['coordinates']
        return (coordinates[0][0][1], coordinates[0][0][0])
    except:
        return None

geos['coordinate'] = geos.apply(fill_coordinates, axis = 1)

In [ ]:
geos.dropna(inplace=True)
geos.drop(columns=['place'], inplace=True)

In [ ]:
coords = geos['coordinate'].tolist()
coords = tuple(coords)

import reverse_geocoder as rg
results = rg.search(coords)

country = []
state = []

for rec in results:
    country.append(rec['cc'])
    state.append(rec['admin1'])

country = pd.Series(country)
state = pd.Series(state)

In [ ]:
geos['country'] = country
geos['state'] = state
geos.dropna(inplace=True)

In [ ]:
from timezonefinder import TimezoneFinder
import datetime
import pytz

tf = TimezoneFinder()

def get_localtime_in_seconds(rows):
    coor = rows['coordinate']
    lati = coor[0]
    logi = coor[1]
    zone_str = tf.timezone_at(lng = logi, lat = lati)
    if zone_str == None:
        return None, None, None, None
    d = datetime.datetime.fromtimestamp(rows['timestamp_ms']/1000, tz=pytz.timezone(zone_str))
    h, m, s = d.hour, d.minute, d.second
    month, weekday, day_in_month = d.month, d.weekday(), d.day
    return h*3600+m*60+s, month, weekday, day_in_month

geos['time_in_second'], geos['month'], geos['weekday'], geos['day_in_month'] = zip(*geos.apply(get_localtime_in_seconds, axis=1))
geos

In [ ]:
geos.dropna(inplace=True)

In [ ]:
def get_label(row):
    t = row['time_in_second']
    # every 15 minute
    return t//(15*60)

geos['label'] = geos.apply(get_label, axis=1)
geos

In [ ]:
# select US only data
geos = geos.loc[geos.country=='US']
geos.reset_index(inplace = True)
geos

In [ ]:
geos

In [ ]:
# Kernel Density Estimate
